# 📘 Final – Visión Computacional - Modalidad Online
---

**Ejercicio práctico final:**  
Para el video que se incluye en esta carpeta es necesario generar otro video donde en el lateral derecho TODO EL TIEMPO se marque si están encendidas las luces del tablero relacionadas a freno de mano, guino izquierda, guino derecha, balizas, luces altas, luces de posición. Además se debe indicar el valor de las revoluciones.

El texto que debe aparecer a la derecha sería:
* Freno de Mano: SI/NO
* Guino Izquierda: SI/NO
* Guino Derecha: SI/NO
* Balizas: SI/NO
* Luces Altas: SI/NO
* Luces Posicion: SI/NO
* R.P.M.: NNNN

No se puede usar YOLO, PYTORCH ni ningún tipo de red neuronal.  
El valor de las revoluciones puede ser aproximado.  
Van a tener que subir el video resultante y el archivo de colab con el cual lo resolvieron.

---

**Autor/es:**  
- Juan Cruz, Rey  

**Profesor:**  
Jorge Martín Acosta

**Institución:**  
Universidad de Palermo

**Fecha:**  
18 de Diciembre 2025

---


## 1. Instalación de Dependencias

In [1]:
%%capture
!pip install opencv-python numpy pillow ipywidgets
!jupyter nbextension enable --py widgetsnbextension

## 2. Importar Librerías

In [2]:
import cv2
import json
import os
import numpy as np
import time
import math
from IPython.display import display
import ipywidgets as widgets
from PIL import Image
from io import BytesIO

## 3. Configuración de Rutas y Colores

In [3]:
# Rutas
VIDEO_PATH = "video.mov"

# Opciones de visualización
MOSTRAR_SOLO_DETECTADAS = True  # True: Solo muestra ROIs cuando estan detectadas | False: Muestra todas siempre
HABILITAR_VISUALIZACION = False   # True: Ejecuta visualizacion en notebook | False: Salta visualizacion

# Colores (BGR)
COLOR_DETECTADO = (0, 255, 255)      # Amarillo
COLOR_ON = (0, 255, 0)               # Verde
COLOR_OFF = (0, 0, 255)              # Rojo
COLOR_INFO = (128, 128, 128)         # Gris


## 4. Configuración de ROIs

**Ejecuta esta celda para definir las áreas de interés (ROIs) y parámetros de detección.**

Este diccionario contiene todas las señales a detectar con sus coordenadas, rangos de color y umbrales de brillo que permitirán detectar las señales.

In [4]:
# Configuración de las ROIs y parámetros de detección
rois_data = {
    "Guino_Izquierda": {
        "nombre_display": "Guino Izquierda",
        "roi": [250, 650, 100, 100],
        "color_rgb": [0, 255, 0],
        "tipo": "parpadeo",
        "color_hsv_bajo": [40, 100, 100],
        "color_hsv_alto": [80, 255, 255],
        "umbral_brillo": 80,
        "descripcion": "Luz verde lateral izquierda - parpadea 2-4s"
    },
    "Guino_Derecha": {
        "nombre_display": "Guino Derecha",
        "roi": [1660, 540, 70, 70],
        "color_rgb": [0, 255, 0],
        "tipo": "parpadeo",
        "color_hsv_bajo": [40, 100, 100],
        "color_hsv_alto": [80, 255, 255],
        "umbral_brillo": 80,
        "descripcion": "Luz verde lateral derecha - parpadea 5-9s"
    },
    "Balizas": {
        "nombre_display": "Balizas",
        "tipo": "combinacion",
        "color_rgb": [255, 255, 0],
        "roi_izq": [180, 520, 60, 60],
        "roi_der": [1140, 420, 60, 60],
        "descripcion": "Ambos guinos parpadeando simultáneamente - 10-13s"
    },
    "Freno_Mano": {
        "nombre_display": "Freno de Mano",
        "roi": [1040, 340, 80, 80],
        "color_rgb": [255, 0, 0],
        "tipo": "luz_fija",
        "color_hsv_bajo": [0, 100, 100],
        "color_hsv_alto": [10, 255, 255],
        "umbral_brillo": 100,
        "descripcion": "Luz roja superior derecha - visible 14-16s"
    },
    "Luces_Altas": {
        "nombre_display": "Luces Altas",
        "roi": [770, 700, 60, 60],
        "color_rgb": [0, 0, 255],
        "tipo": "luz_fija",
        "color_hsv_bajo": [100, 150, 100],
        "color_hsv_alto": [130, 255, 255],
        "umbral_brillo": 80,
        "descripcion": "Luz azul inferior central - parpadea 17-24s"
    },
    "Luces_Posicion": {
        "nombre_display": "Luces Posicion",
        "roi": [680, 740, 60, 50],
        "color_rgb": [0, 255, 0],
        "tipo": "luz_fija",
        "color_hsv_bajo": [40, 100, 100],
        "color_hsv_alto": [80, 255, 255],
        "umbral_brillo": 80,
        "descripcion": "Luz verde inferior central - parpadea 25-28s"
    },
    "RPM": {
        "nombre_display": "R.P.M.",
        "roi_completa": [470, 550, 300, 300],
        "roi_centro": [320, 420, 60, 60],
        "color_rgb": [255, 165, 0],
        "tipo": "aguja_angular",
        "color_aguja_hsv_bajo": [12, 100, 100],
        "color_aguja_hsv_alto": [22, 255, 255],
        "descripcion": "Tacómetro 0-4000 RPM - Gira anti-horario, ángulo disminuye: 248° (0 RPM) hasta 164° (4000 RPM, marca '4')"
    }
}

print(f"ROIs configuradas: {len(rois_data)} señales")
print("\nSeñales disponibles:")
for nombre, datos in rois_data.items():
    nombre_display = datos.get('nombre_display', nombre)
    print(f"  - {nombre_display}")

ROIs configuradas: 7 señales

Señales disponibles:
  - Guino Izquierda
  - Guino Derecha
  - Balizas
  - Freno de Mano
  - Luces Altas
  - Luces Posicion
  - R.P.M.


## 5. Verificar que exista video Local

El video `video.mov` debe estar en el mismo directorio que este notebook para poder ejecutar el script.

In [5]:
import os

if os.path.exists(VIDEO_PATH):
    print(f"✅ Video encontrado: {VIDEO_PATH}")

    # Obtener información del video
    cap = cv2.VideoCapture(VIDEO_PATH)
    if cap.isOpened():
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duracion = total_frames / fps if fps > 0 else 0

        print(f"   Resolución: {width}x{height}")
        print(f"   FPS: {fps}")
        print(f"   Duración: {duracion:.1f} segundos")
        print(f"   Total frames: {total_frames}")
        cap.release()
    else:
        print(f"⚠️  No se pudo abrir el video")
else:
    print(f"❌ Video no encontrado: {VIDEO_PATH}")
    print(f"   Asegúrate de que el archivo esté en: {os.path.abspath(VIDEO_PATH)}")

✅ Video encontrado: video.mov
   Resolución: 1920x1080
   FPS: 29.987282746926663
   Duración: 39.3 segundos
   Total frames: 1179


## 6. Funciones de Detección

In [6]:
def detectar_senal_en_roi(roi_frame, datos_roi):
    """Detecta si hay una señal activa en la ROI"""
    if datos_roi.get("tipo") == "combinacion":
        return False

    # El RPM se maneja por separado
    if datos_roi.get("tipo") == "aguja_angular":
        return False

    umbral_brillo = datos_roi.get("umbral_brillo", 80)
    hsv = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2HSV)

    # Si hay rangos HSV definidos, SOLO usar detección por color (más precisa)
    if "color_hsv_bajo" in datos_roi and "color_hsv_alto" in datos_roi:
        hsv_bajo = np.array(datos_roi["color_hsv_bajo"])
        hsv_alto = np.array(datos_roi["color_hsv_alto"])
        mask = cv2.inRange(hsv, hsv_bajo, hsv_alto)
        porcentaje_color = (np.sum(mask > 0) / mask.size) * 100

        # Requiere al menos 5% de píxeles con el color correcto
        return porcentaje_color > 5

    # Solo si NO hay rangos HSV, usar detección por brillo (fallback)
    gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
    brillo_medio = np.mean(gray)

    return brillo_medio > umbral_brillo


def detectar_todas_las_senales(frame, rois_data):
    """Detecta el estado de todas las señales en el frame"""
    estados_deteccion = {}

    for nombre, datos in rois_data.items():
        if datos.get("tipo") == "combinacion":
            continue

        # Manejar RPM por separado
        if datos.get("tipo") == "aguja_angular" and nombre == "RPM":
            rpm_actual = detectar_rpm_tacometro(frame, datos)
            # Marcar como detectado si hay movimiento (RPM > umbral)
            estados_deteccion[nombre] = rpm_tiene_movimiento(rpm_actual)
            # Guardar el valor RPM para mostrarlo
            estados_deteccion[f"{nombre}_valor"] = rpm_actual
            continue

        roi = obtener_coordenadas_roi(datos)
        if roi is None:
            continue

        x, y, w, h = roi
        roi_frame = frame[y:y+h, x:x+w]

        if roi_frame.size > 0:
            estados_deteccion[nombre] = detectar_senal_en_roi(roi_frame, datos)

    return estados_deteccion


def detectar_balizas(estados_deteccion):
    """Detecta si las balizas están activas (ambos guinos encendidos)"""
    return (estados_deteccion.get("Guino_Izquierda", False) and
            estados_deteccion.get("Guino_Derecha", False))

In [7]:
# Configuración del tacómetro
RPM_MAX_VALUE = 4000   # En este video, la aguja solo llega hasta la marca "4" (4000 RPM)
RPM_START_ANGLE = 248  # Ángulo en grados para 0 RPM (reposo)
RPM_END_ANGLE = 164    # Ángulo en grados para 4000 RPM (máximo observado en el video)
RPM_CENTER_X = 670     # Centro X del tacómetro
RPM_CENTER_Y = 680     # Centro Y del tacómetro
RPM_UMBRAL_MINIMO = 300  # RPM mínimo para considerar que hay movimiento


def detectar_rpm_tacometro(frame, datos_roi):
    """
    Detecta el valor de RPM analizando el ángulo de la aguja

    El tacómetro gira en sentido ANTI-HORARIO (ángulo disminuye al aumentar RPM):
    - 0 RPM: ~248° (aguja en reposo, arriba-izquierda)
    - 4000 RPM: ~164° (aguja al máximo en este video, cerca de la marca "4")
    - Rango total: 84° de rotación

    Nota: El tacómetro físico tiene marcas hasta 8 (8000 RPM), pero en este
    video la aguja solo llega hasta aproximadamente la marca 4 (4000 RPM).

    Args:
        frame: Frame del video completo
        datos_roi: Datos de configuración del ROI del tacómetro

    Returns:
        int: Valor de RPM detectado (0-4000)
    """
    # Obtener ROI completa del tacómetro (solo contiene la aguja)
    roi_coords = datos_roi.get("roi_completa")
    if roi_coords is None:
        return 0

    x_rpm, y_rpm, w_rpm, h_rpm = roi_coords
    gauge_img = frame[y_rpm:y_rpm+h_rpm, x_rpm:x_rpm+w_rpm].copy()

    # Convertir a HSV para detectar la aguja naranja
    hsv = cv2.cvtColor(gauge_img, cv2.COLOR_BGR2HSV)

    # Máscara para detectar aguja naranja
    hsv_bajo = np.array(datos_roi.get("color_aguja_hsv_bajo", [12, 100, 100]))
    hsv_alto = np.array(datos_roi.get("color_aguja_hsv_alto", [22, 255, 255]))

    mask = cv2.inRange(hsv, hsv_bajo, hsv_alto)

    # Encontrar contornos
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return 0

    # SIMPLIFICADO: Como el ROI solo contiene la aguja,
    # tomamos el contorno más grande
    contorno_aguja = max(contours, key=cv2.contourArea)

    # Si el contorno es muy pequeño, probablemente es ruido
    if cv2.contourArea(contorno_aguja) < 20:
        return 0

    # Centro del tacómetro (relativo al ROI)
    center_x_roi = RPM_CENTER_X - x_rpm
    center_y_roi = RPM_CENTER_Y - y_rpm

    # Encontrar el punto más alejado del centro (punta de la aguja)
    max_dist = 0
    tip_x_roi = center_x_roi
    tip_y_roi = center_y_roi

    for point in contorno_aguja:
        px, py = point[0]
        dist = math.sqrt((px - center_x_roi)**2 + (py - center_y_roi)**2)
        if dist > max_dist:
            max_dist = dist
            tip_x_roi = px
            tip_y_roi = py

    # Calcular ángulo usando atan2
    angle_rad = math.atan2(center_y_roi - tip_y_roi, tip_x_roi - center_x_roi)
    angle_deg = (angle_rad * 180 / math.pi) % 360

    # Mapear ángulo a RPM (ángulo DISMINUYE al aumentar RPM: 248° -> 164°)
    # Mayor ángulo = menor RPM
    angle_from_max = RPM_START_ANGLE - angle_deg  # Distancia desde 0 RPM
    angle_span = RPM_START_ANGLE - RPM_END_ANGLE  # 84° total

    # Calcular RPM con interpolación lineal
    if angle_from_max <= 0:
        # Ángulo >= START_ANGLE, estamos en 0 RPM o menos
        rpm = 0
    elif angle_from_max >= angle_span:
        # Ángulo <= END_ANGLE, estamos en RPM máximo o más
        rpm = RPM_MAX_VALUE
    else:
        # Interpolación lineal
        rpm = int((angle_from_max / angle_span) * RPM_MAX_VALUE)

    return rpm


rpm_anterior = 0

def rpm_tiene_movimiento(rpm_actual, umbral_minimo=RPM_UMBRAL_MINIMO):
    """
    Determina si hay movimiento en el tacómetro

    Args:
        rpm_actual: Valor actual de RPM
        umbral_minimo: RPM mínimo para considerar que hay movimiento

    Returns:
        bool: True si RPM > umbral_minimo (indica que la aguja se movió)
    """
    global rpm_anterior

    # Hay movimiento si el RPM es mayor al umbral mínimo
    hay_movimiento = rpm_actual > umbral_minimo

    # Actualizar RPM anterior
    rpm_anterior = rpm_actual

    return hay_movimiento

## 7. Funciones de Utilidad

In [8]:
def obtener_coordenadas_roi(datos_roi):
    """Obtiene las coordenadas (x, y, w, h) de una ROI"""
    if "roi" in datos_roi and datos_roi["roi"] is not None:
        return datos_roi["roi"]
    elif "roi_completa" in datos_roi:
        return datos_roi["roi_completa"]
    return None


def obtener_color_roi(datos_roi):
    """Obtiene el color BGR de una ROI desde el JSON"""
    if "color_rgb" in datos_roi and datos_roi["color_rgb"] is not None:
        rgb = datos_roi["color_rgb"]
        return (rgb[2], rgb[1], rgb[0])  # Convertir RGB a BGR
    elif "color_bgr" in datos_roi and datos_roi["color_bgr"] is not None:
        return tuple(datos_roi["color_bgr"])
    # Blanco por defecto (nunca debería llegar aquí)
    return (255, 255, 255)


def preparar_lista_estados(estados_deteccion, balizas_activas, rois_data):
    """Prepara la lista de estados de todas las señales para mostrar"""
    lista_estados = []

    for nombre_tecnico, datos in rois_data.items():
        nombre_display = datos.get("nombre_display", nombre_tecnico)

        if nombre_tecnico == "Balizas":
            # Balizas: mostrar su estado real
            lista_estados.append((nombre_display, balizas_activas))
        elif nombre_tecnico in ["Guino_Izquierda", "Guino_Derecha"]:
            # Guiños: si hay balizas activas, mostrar OFF
            # Si NO hay balizas, mostrar su estado real
            if balizas_activas:
                lista_estados.append((nombre_display, False))
            else:
                estado = estados_deteccion.get(nombre_tecnico, False)
                lista_estados.append((nombre_display, estado))
        elif nombre_tecnico == "RPM":
            estado = estados_deteccion.get(nombre_tecnico, False)
            rpm_valor = estados_deteccion.get(f"{nombre_tecnico}_valor", 0)
            # RPM tiene formato especial: (nombre, estado, valor_rpm)
            lista_estados.append((nombre_display, estado, rpm_valor))
        else:
            estado = estados_deteccion.get(nombre_tecnico, False)
            lista_estados.append((nombre_display, estado))

    return lista_estados

## 8. Funciones de Visualización

In [ ]:
def dibujar_roi(frame, nombre, datos_roi, estado_detectado, balizas_activas):
    """Dibuja una ROI individual en el frame"""
    # Si la bandera está activa, solo dibujar si está detectada
    if MOSTRAR_SOLO_DETECTADAS and not estado_detectado:
        return

    roi = obtener_coordenadas_roi(datos_roi)
    if roi is None:
        return

    x, y, w, h = roi
    color_normal = obtener_color_roi(datos_roi)
    color = COLOR_DETECTADO if estado_detectado else color_normal
    grosor = 4 if estado_detectado else 2

    cv2.rectangle(frame, (x, y), (x+w, y+h), color, grosor)

    # Etiqueta
    if balizas_activas and nombre in ["Guino_Izquierda", "Guino_Derecha"]:
        etiqueta = "BALIZAS [DETECTADO]"
    elif estado_detectado:
        etiqueta = f"{nombre} [DETECTADO]"
    else:
        etiqueta = nombre

    cv2.putText(frame, etiqueta, (x, y-10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


def dibujar_todas_las_rois(frame, rois_data, estados_deteccion, balizas_activas):
    """Dibuja todas las ROIs en el frame"""
    for nombre, datos in rois_data.items():
        if datos.get("tipo") == "combinacion":
            continue

        estado_detectado = estados_deteccion.get(nombre, False)
        dibujar_roi(frame, nombre, datos, estado_detectado, balizas_activas)


def dibujar_panel_estados(frame, lista_estados):
    """Dibuja el panel de estados en el lateral derecho"""
    # Obtener dimensiones del frame
    frame_height, frame_width = frame.shape[:2]

    line_height = 30
    font_scale = 0.6
    font_thickness = 2
    padding = 10

    # Calcular tamaño del panel
    max_width = 0
    for item in lista_estados:
        # Verificar si es RPM (tupla de 3 elementos)
        if len(item) == 3:
            nombre_display, _, rpm_valor = item
            texto = f"{nombre_display}: {rpm_valor}"
        else:
            nombre_display = item[0]
            texto = f"{nombre_display}: NO"

        (text_w, _), _ = cv2.getTextSize(texto, cv2.FONT_HERSHEY_SIMPLEX,
                                         font_scale, font_thickness)
        max_width = max(max_width, text_w)

    panel_width = max_width + padding * 2
    panel_height = len(lista_estados) * line_height + padding * 2

    # POSICION A LA DERECHA
    panel_x = frame_width - panel_width - 10  # 10px desde el borde derecho
    panel_y = 10

    # Fondo negro
    cv2.rectangle(frame,
                 (panel_x, panel_y),
                 (panel_x + panel_width, panel_y + panel_height),
                 (0, 0, 0), -1)

    # Dibujar líneas de estado
    for i, item in enumerate(lista_estados):
        y_pos = panel_y + padding + (i + 1) * line_height - 5

        # Verificar si es RPM (tupla de 3 elementos)
        if len(item) == 3:
            nombre_display, estado, rpm_valor = item
            color_estado = COLOR_ON if estado else COLOR_OFF
            texto = f"{nombre_display}: {rpm_valor}"  # Sin duplicar RPM
        else:
            nombre_display, estado = item
            color_estado = COLOR_ON if estado else COLOR_OFF
            estado_texto = "SI" if estado else "NO"  # Cambiado de ON/OFF a SI/NO
            texto = f"{nombre_display}: {estado_texto}"

        cv2.putText(frame, texto, (panel_x + padding, y_pos),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale, color_estado, font_thickness)




def dibujar_tiempo_video(frame, panel_height, frame_actual, fps, duracion):
    """Dibuja el tiempo actual del video en el lateral derecho, debajo del panel"""
    # Obtener dimensiones del frame
    frame_height, frame_width = frame.shape[:2]

    tiempo_actual = frame_actual / fps if fps > 0 else 0
    tiempo_text = f"Tiempo: {tiempo_actual:.1f}s / {duracion:.1f}s"

    # Configuración mejorada
    font_scale = 0.65
    font_thickness = 2
    padding = 8

    # Calcular tamaño del texto
    (text_w, text_h), _ = cv2.getTextSize(tiempo_text, cv2.FONT_HERSHEY_SIMPLEX,
                                           font_scale, font_thickness)

    # Posición a la derecha, debajo del panel
    info_width = text_w + padding * 2
    info_height = text_h + padding * 2 + 10

    panel_x = frame_width - info_width - 10  # Alineado con el panel derecho
    panel_y = 10
    info_y = panel_y + panel_height + 10  # Debajo del panel de estados

    # Fondo negro
    cv2.rectangle(frame,
                 (panel_x, info_y),
                 (panel_x + info_width, info_y + info_height),
                 (0, 0, 0), -1)

    # Borde gris sutil para mejor definición
    cv2.rectangle(frame,
                 (panel_x, info_y),
                 (panel_x + info_width, info_y + info_height),
                 (80, 80, 80), 1)

    # Texto blanco para mejor contraste
    cv2.putText(frame, tiempo_text,
               (panel_x + padding, info_y + text_h + padding + 5),
               cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)

## 9. Función Principal de Procesamiento

In [10]:
def procesar_frame(frame, rois_data, frame_actual, fps, duracion):
    """Procesa un frame individual y retorna el frame con las detecciones dibujadas"""
    # Detectar señales
    estados_deteccion = detectar_todas_las_senales(frame, rois_data)
    balizas_activas = detectar_balizas(estados_deteccion)

    # Dibujar elementos
    dibujar_todas_las_rois(frame, rois_data, estados_deteccion, balizas_activas)

    lista_estados = preparar_lista_estados(estados_deteccion, balizas_activas, rois_data)
    dibujar_panel_estados(frame, lista_estados)

    panel_height = len(lista_estados) * 30 + 20
    dibujar_tiempo_video(frame, panel_height, frame_actual, fps, duracion)

    return frame

## 10. Visualización en Jupyter Local

Esta función muestra el video frame por frame en el notebook.

**Nota**: En Jupyter local, la visualización no será en tiempo real como en Colab, pero puedes controlar la velocidad de reproducción.

In [11]:
def visualizar_video_notebook(video_path, rois_data, duracion_seg=None, inicio_seg=0):
    """
    Visualiza el video en el notebook con detecciones

    Args:
        video_path: Ruta al video
        rois_data: Datos de las ROIs
        duracion_seg: Cuántos segundos del video mostrar (None = hasta el final)
        inicio_seg: Desde qué segundo empezar (0 = desde el inicio)
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"[ERROR] No se pudo abrir el video: {video_path}")
        return

    # Información del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duracion_total = total_frames / fps if fps > 0 else 0

    # Calcular frame inicial
    frame_inicio = int(inicio_seg * fps)

    # Posicionar el video en el frame inicial
    if frame_inicio > 0:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_inicio)

    # Calcular frames a procesar
    if duracion_seg:
        max_frames = frame_inicio + int(duracion_seg * fps)
    else:
        max_frames = total_frames

    print("="*70)
    print("VISUALIZADOR DE VIDEO")
    print("="*70)
    print(f"Video: {video_path}")
    print(f"FPS: {fps}")
    print(f"Duración total: {duracion_total:.1f} segundos")
    if inicio_seg > 0:
        print(f"Inicio: {inicio_seg:.1f}s")
    if duracion_seg:
        print(f"Mostrando: {duracion_seg:.1f} segundos")
    else:
        print(f"Mostrando: hasta el final ({duracion_total - inicio_seg:.1f}s)")
    print("="*70)
    print("Procesando video...")
    print("="*70)

    # Widget para mostrar el video
    image_widget = widgets.Image(format='jpeg', width=800)
    display(image_widget)

    frame_actual = frame_inicio
    tiempo_inicio = time.time()

    try:
        while frame_actual < max_frames:
            ret, frame = cap.read()

            if not ret:
                print("\n[INFO] Fin del video alcanzado")
                break

            # Procesar frame
            frame_procesado = procesar_frame(frame, rois_data, frame_actual, fps, duracion_total)

            # Convertir BGR a RGB para visualización
            frame_rgb = cv2.cvtColor(frame_procesado, cv2.COLOR_BGR2RGB)

            # Convertir a JPEG para mostrar en el widget
            pil_img = Image.fromarray(frame_rgb)
            buffer = BytesIO()
            pil_img.save(buffer, format='JPEG', quality=85)
            image_widget.value = buffer.getvalue()

            frame_actual += 1

            # Control de velocidad - actualizar cada N frames para que sea más fluido
            # En Jupyter local, mostrar cada 2 frames para mejor rendimiento
            if frame_actual % 2 == 0:
                time.sleep(1/fps)  # Pausar según FPS para simular tiempo real

    except KeyboardInterrupt:
        print("\n[INFO] Reproducción interrumpida")
    except Exception as e:
        print(f"\n[ERROR] Error durante la reproducción: {e}")
    finally:
        cap.release()
        tiempo_total = time.time() - tiempo_inicio
        print(f"\n[OK] Reproducción finalizada")
        print(f"Frames procesados: {frame_actual - frame_inicio}")
        print(f"Tiempo total: {tiempo_total:.1f} segundos")

## 11. ▶️ Ejecutar Visualización

**Ejecuta esta celda para ver el video con las detecciones.**

Puedes interrumpir la ejecución en cualquier momento con el botón "Stop" de Jupyter.

In [12]:
# Verificar si la visualización está habilitada
if HABILITAR_VISUALIZACION:
    visualizar_video_notebook(VIDEO_PATH, rois_data)
else:
    print("[INFO] Visualizacion deshabilitada (HABILITAR_VISUALIZACION = False)")
    print("       Ejecuta la celda de exportacion para generar el video.")

[INFO] Visualizacion deshabilitada (HABILITAR_VISUALIZACION = False)
       Ejecuta la celda de exportacion para generar el video.


## 12. Exportar Video

Esta seccion permite generar el video resultante con todas las detecciones.

**IMPORTANTE:** Esta es la funcion principal del trabajo practico. Genera el video final para entregar.

In [13]:
def exportar_video_procesado(video_path, rois_data, output_path="video_procesado.mp4"):
    """Exporta el video procesado a un archivo"""
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"[ERROR] No se pudo abrir el video: {video_path}")
        return

    # Obtener propiedades del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duracion = total_frames / fps if fps > 0 else 0

    # Crear writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    print("="*70)
    print("EXPORTACION DE VIDEO")
    print("="*70)
    print(f"Video de entrada: {video_path}")
    print(f"Video de salida: {output_path}")
    print(f"Resolucion: {width}x{height}")
    print(f"FPS: {fps}")
    print(f"Duracion: {duracion:.1f} segundos")
    print(f"Total frames: {total_frames}")
    print("="*70)
    print("Procesando...")
    print("="*70)

    frame_actual = 0

    try:
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # Procesar frame
            frame_procesado = procesar_frame(frame, rois_data, frame_actual, fps, duracion)

            # Escribir frame
            out.write(frame_procesado)

            frame_actual += 1

            # Mostrar progreso cada 30 frames
            if frame_actual % 30 == 0:
                progreso = (frame_actual / total_frames) * 100
                print(f"Progreso: {progreso:.1f}% ({frame_actual}/{total_frames} frames)", end="\r")

    finally:
        cap.release()
        out.release()
        print("\n")
        print("="*70)
        print("[OK] Video exportado exitosamente")
        print(f"Archivo: {output_path}")
        print(f"Frames procesados: {frame_actual}")
        print("="*70)


In [14]:
# Ejecutar exportacion del video completo
exportar_video_procesado(VIDEO_PATH, rois_data, "video_procesado.mp4")

EXPORTACION DE VIDEO
Video de entrada: video.mov
Video de salida: video_procesado.mp4
Resolucion: 1920x1080
FPS: 29.987282746926663
Duracion: 39.3 segundos
Total frames: 1179
Procesando...


[OK] Video exportado exitosamente
Archivo: video_procesado.mp4
Frames procesados: 1179
